In [1]:
import ast
import glob
import json
import multiprocessing as mp
import nltk
import numpy as np
import pandas as pd
import string
import textacy

from collections import Counter, defaultdict
from gensim.models.word2vec import Word2Vec
from IPython.core.debugger import Pdb
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [2]:
%matplotlib inline

In [3]:
pdb = Pdb()

pool = mp.Pool(processes=mp.cpu_count())

sw = stopwords.words('english')

In [4]:
df = pd.read_csv('./Electronics_5.csv')
meta_df = pd.read_csv('./meta_electronics.csv')

In [12]:
products_with_titles = meta_df[meta_df.title.notnull()]

In [3]:
trans = str.maketrans(string.punctuation, ' '*len(string.punctuation))

def tokenize(sent):                                                    
    sent = sent.translate(trans)
    return [w.lower() for w in word_tokenize(sent) if w.lower() not in sw]


In [4]:
#w2v = Word2Vec([tokenize(review) for review in df.reviewText], min_count=1000, workers=6)

In [6]:
# wv = w2v.wv
# del w2v

In [2]:
#w2v.save('./w2v_reviews.model')
wv = Word2Vec.load('./w2v_reviews.model')

NameError: name 'Word2Vec' is not defined

In [1]:
wv.most_similar(positive=['the', 'awful', 'sound'])

NameError: name 'wv' is not defined

In [29]:
df.asin[(df.reviewText.notnull()) & (df.reviewText.str.contains('tinny'))].value_counts()

B003ELYQGG    59
B0038W0K2K    39
B0027VT6V4    37
B000R9AAJA    34
B000RQSGYO    30
B001V9LPT4    27
B009A5204K    20
B0052YFYFK    17
B000I68BD4    17
B004HHICKC    17
B0082E9K7U    17
B0047T79VS    15
B000E6G9RI    13
B005A0B7CA    13
B007OY5V68    13
B003GIJTR8    12
B003VQQVDU    12
B004S7Q8CA    12
B001ELK2NY    12
B002BH3I9U    12
B0055QYJM4    12
B001UEBN42    11
B00A0JUNYY    11
B000ZH98LU    11
B00020S7XK    10
B002RWJD7A    10
B001EZYMF4    10
B008JGR9MO    10
B000IF4788    10
B002ZAVVHY    10
              ..
B002DEM9EG     1
B002E95P3C     1
B004477OAY     1
B0036VO2B8     1
B003WZFYYW     1
B000G36GOK     1
B0015BUJ0Y     1
B003JTHMRS     1
B005UA65YA     1
B001DJ64D4     1
B004MEWZE4     1
B004P5ZAQK     1
B0009GUDVQ     1
B003ZUXITC     1
B0057153KY     1
B004MNADXK     1
B0000U19F0     1
B0006DPPW4     1
B00BB0ZTJA     1
B0009X09IQ     1
B000EPHP4U     1
B00009P2YQ     1
B003OTXBY6     1
B0074YRGIY     1
B0016BVDIK     1
B002MV2R5G     1
B00CE590Z0     1
1400599997    

In [11]:
products = df.groupby('asin')

df_rev_gt1000 = products.filter(lambda x: len(x) > 1000)
print(df_rev_gt1000)

               asin   helpful  overall  \
14790    B00004ZCJE    [1, 1]      4.0   
14791    B00004ZCJE    [0, 0]      3.0   
14792    B00004ZCJE    [0, 0]      5.0   
14793    B00004ZCJE    [0, 0]      5.0   
14794    B00004ZCJE    [0, 0]      3.0   
14795    B00004ZCJE    [0, 0]      5.0   
14796    B00004ZCJE    [0, 0]      5.0   
14797    B00004ZCJE    [0, 0]      4.0   
14798    B00004ZCJE    [1, 1]      4.0   
14799    B00004ZCJE    [0, 0]      4.0   
14800    B00004ZCJE  [18, 19]      4.0   
14801    B00004ZCJE    [0, 0]      5.0   
14802    B00004ZCJE    [0, 0]      5.0   
14803    B00004ZCJE    [0, 0]      2.0   
14804    B00004ZCJE    [0, 0]      5.0   
14805    B00004ZCJE    [0, 0]      5.0   
14806    B00004ZCJE    [0, 1]      5.0   
14807    B00004ZCJE    [0, 0]      1.0   
14808    B00004ZCJE    [0, 0]      5.0   
14809    B00004ZCJE    [0, 0]      5.0   
14810    B00004ZCJE  [22, 24]      3.0   
14811    B00004ZCJE  [10, 11]      3.0   
14812    B00004ZCJE    [0, 0]     

helpful           B000I68BD4
overall           B000I68BD4
reviewText        B000I68BD4
reviewTime        B000I68BD4
reviewerID        B000I68BD4
reviewerName      B000I68BD4
summary           B000I68BD4
unixReviewTime    B000I68BD4
dtype: object

In [15]:
df_rev_gt1000.groupby('asin').apply(lambda x: x.overall.std()).argmax()

'B000I68BD4'

In [53]:
test_product = df_rev_gt1000[df_rev_gt1000.asin == 'B000I68BD4']

In [54]:
c = Counter()
for review in test_product.reviewText:
    c.update(set(tokenize(review)))

In [55]:
c.most_common(25)

[('sound', 640),
 ('ear', 454),
 ('good', 408),
 ('buds', 376),
 ('price', 339),
 ('quality', 327),
 ('great', 318),
 ('noise', 247),
 ('like', 237),
 ('ears', 231),
 ('use', 231),
 ('better', 227),
 ('comfortable', 225),
 ('earbuds', 221),
 ('fit', 207),
 ('headphones', 205),
 ('pair', 204),
 ('get', 194),
 ('would', 193),
 ('well', 186),
 ('bought', 180),
 ('much', 179),
 ('one', 178),
 ('music', 142),
 ('work', 141)]

In [30]:
list(textacy.extract.noun_chunks(textacy.Doc(' '.join(tokenize(test_product.iloc[1].reviewText)))))

[little ear buds,
 lot business trips,
 bring along ear buds,
 better sound quality,
 2 bucks,
 inexpensive great sound,
 nearly year failures]

In [146]:
adj_noun_c = Counter()
n_chunk_c = Counter()

for review in test_product.reviewText:
    doc = textacy.Doc(review)
    adj_noun_c.update(set([str(w) for w in textacy.extract.words(doc, filter_stops=True, filter_nums=True, 
                                                             filter_punct=True, include_pos=['ADJ', 'NOUN'])]))
    
    n_chunk_c.update(set([str(' '.join(str(nc).split()[-2:])).lower() for nc in textacy.extract.noun_chunks(doc) if str(nc).lower() not in sw and len(str(nc).split(' ')) > 1]))
    

In [124]:
adj_noun_c.most_common(50)

[('sound', 535),
 ('ear', 432),
 ('good', 382),
 ('buds', 352),
 ('price', 330),
 ('quality', 319),
 ('great', 272),
 ('noise', 230),
 ('ears', 224),
 ('comfortable', 213),
 ('earbuds', 204),
 ('pair', 204),
 ('headphones', 200),
 ('better', 189),
 ('music', 138),
 ('cheap', 128),
 ('bass', 120),
 ('time', 109),
 ('set', 104),
 ('ones', 104),
 ('different', 98),
 ('cord', 93),
 ('product', 87),
 ('months', 86),
 ('fit', 84),
 ('little', 83),
 ('earphones', 83),
 ('best', 74),
 ('money', 73),
 ('volume', 68),
 ('use', 65),
 ('nice', 64),
 ('worth', 63),
 ('reviews', 63),
 ('bud', 63),
 ('bad', 62),
 ('new', 60),
 ('decent', 60),
 ('small', 60),
 ('bit', 60),
 ('sizes', 60),
 ('low', 59),
 ('reduction', 58),
 ('high', 58),
 ('size', 58),
 ('pairs', 56),
 ('right', 55),
 ('rubber', 55),
 ('year', 53),
 ('lot', 53)]

In [147]:
n_chunk_c.most_common(50)

[('ear buds', 239),
 ('sound quality', 181),
 ('my ears', 144),
 ('my ear', 49),
 ('noise reduction', 45),
 ('mp3 player', 40),
 ('your ears', 37),
 ('my ipod', 36),
 ('your ear', 35),
 ('ear bud', 26),
 ('my music', 22),
 ('ear pieces', 20),
 ('price point', 19),
 ('second pair', 19),
 ('outside noise', 18),
 ('low price', 18),
 ('different sizes', 18),
 ('quality sound', 17),
 ('ear canal', 17),
 ('background noise', 17),
 ('first pair', 16),
 ('my wife', 16),
 ('great price', 14),
 ('6 months', 14),
 ('good price', 13),
 ('mp3 players', 13),
 ('your money', 13),
 ('price range', 12),
 ('my daughter', 12),
 ('one side', 12),
 ('head phones', 12),
 ('ear phones', 12),
 ('my son', 11),
 ('bass response', 11),
 ('my computer', 11),
 ('cheap headphones', 11),
 ('great product', 11),
 ('ambient noise', 11),
 ('few months', 11),
 ('other earbuds', 11),
 ('great value', 11),
 ('rubber tips', 11),
 ('good job', 11),
 ('external noise', 11),
 ('3 sizes', 11),
 ('new pair', 11),
 ('two pairs',

In [156]:
wv.most_similar(['different', 'sizes'], topn=25)

[('styles', 0.6896075010299683),
 ('types', 0.650073766708374),
 ('inserts', 0.5676872134208679),
 ('combinations', 0.5588642358779907),
 ('sets', 0.5392212867736816),
 ('tips', 0.5151994228363037),
 ('attachments', 0.5132334232330322),
 ('varying', 0.49580249190330505),
 ('various', 0.487615168094635),
 ('assortment', 0.48510217666625977),
 ('configurations', 0.4735668897628784),
 ('earpieces', 0.46533215045928955),
 ('variety', 0.44576239585876465),
 ('choices', 0.44565796852111816),
 ('individual', 0.44294118881225586),
 ('ways', 0.4386582374572754),
 ('kinds', 0.43704819679260254),
 ('adaptors', 0.43534374237060547),
 ('interchangeable', 0.4307205080986023),
 ('adapters', 0.4297689199447632),
 ('positions', 0.426310658454895),
 ('choose', 0.423108845949173),
 ('largest', 0.4195912778377533),
 ('pairs', 0.41837209463119507),
 ('individually', 0.4109639525413513)]

In [137]:
wv.most_similar('retailer')

[('retailers', 0.7870277166366577),
 ('dealer', 0.7712366580963135),
 ('bestbuy', 0.7514880895614624),
 ('costco', 0.7403971552848816),
 ('discount', 0.7256439328193665),
 ('staples', 0.7133115530014038),
 ('stores', 0.7011342644691467),
 ('walmart', 0.6890081167221069),
 ('store', 0.675402045249939),
 ('sale', 0.6643613576889038)]

In [182]:
mc_100 = n_chunk_c.most_common(100)
mc_100

[('ear buds', 239),
 ('sound quality', 181),
 ('my ears', 144),
 ('my ear', 49),
 ('noise reduction', 45),
 ('mp3 player', 40),
 ('your ears', 37),
 ('my ipod', 36),
 ('your ear', 35),
 ('ear bud', 26),
 ('my music', 22),
 ('ear pieces', 20),
 ('price point', 19),
 ('second pair', 19),
 ('outside noise', 18),
 ('low price', 18),
 ('different sizes', 18),
 ('quality sound', 17),
 ('ear canal', 17),
 ('background noise', 17),
 ('first pair', 16),
 ('my wife', 16),
 ('great price', 14),
 ('6 months', 14),
 ('good price', 13),
 ('mp3 players', 13),
 ('your money', 13),
 ('price range', 12),
 ('my daughter', 12),
 ('one side', 12),
 ('head phones', 12),
 ('ear phones', 12),
 ('my son', 11),
 ('bass response', 11),
 ('my computer', 11),
 ('cheap headphones', 11),
 ('great product', 11),
 ('ambient noise', 11),
 ('few months', 11),
 ('other earbuds', 11),
 ('great value', 11),
 ('rubber tips', 11),
 ('good job', 11),
 ('external noise', 11),
 ('3 sizes', 11),
 ('new pair', 11),
 ('two pairs',

In [185]:
ratings = defaultdict(int)
for rating, review in zip(test_product.overall, test_product.reviewText):
    d = textacy.Doc(review)
    for term, count in mc_100:
        if term in list(textacy.extract.noun_chunks(d)):
            ratings[str(term)] += rating - 3

TypeError: Argument 'other' has incorrect type (expected spacy.tokens.span.Span, got str)

In [181]:
sorted([(term, ratings[term]/int(count)) for term, count in mc_100], key=lambda x: x[1], reverse=True)

[('bud', 26.600000000000001),
 ('one', 16.199999999999999),
 ('ear', 11.938775510204081),
 ('comfort', 11.695652173913043),
 ('fit', 11.25),
 ('use', 9.6071428571428577),
 ('ones', 8.6666666666666661),
 ('ears', 8.137931034482758),
 ('ear bud', 7.7692307692307692),
 ('purchase', 6.0769230769230766),
 ('work', 5.9705882352941178),
 ('day', 4.5384615384615383),
 ('year', 4.4500000000000002),
 ('buds', 4.1739130434782608),
 ('my ear', 3.6122448979591835),
 ('quality', 3.403225806451613),
 ('set', 3.2105263157894739),
 ('way', 3.0476190476190474),
 ('job', 2.8461538461538463),
 ('color', 2.7857142857142856),
 ('time', 2.7708333333333335),
 ('type', 2.5),
 ('plug', 2.4705882352941178),
 ('noise', 2.2873563218390807),
 ('outside noise', 2.0),
 ('audiophile', 1.962962962962963),
 ('case', 1.8888888888888888),
 ('reason', 1.875),
 ('ipod', 1.6499999999999999),
 ('fact', 1.6363636363636365),
 ('earphones', 1.6000000000000001),
 ('sound', 1.5725190839694656),
 ('others', 1.5714285714285714),
 ('

In [178]:
%debug

> <ipython-input-177-94f3ea313f9a>(2)<listcomp>()
      1 mc_100_dict = dict(mc_100)
----> 2 [(term, rating[term]/int(count)) for term, count in mc_100]

ipdb> term
'sound'
ipdb> count
262
ipdb> type(term)
<class 'str'>
ipdb> rating[term]
*** IndexError: invalid index to scalar variable.
ipdb> q
